In [ ]:
import pandas as pd
import time
import pickle
import logging
import sys
from datetime import datetime
from matplotlib import pyplot as plt
import warnings
import argparse

# from sklearn.metrics import mean_squared_error
# import numpy as np
import os
import seaborn as sns

In [ ]:
file_datetime = "2025-03-19_10-47-56"
filename_FB = f"../data/nextbike/hourly_demand_supply_Freiburg {file_datetime}.csv"
filename_DD = f"../data/nextbike/hourly_demand_supply_Dresden {file_datetime}.csv"


df_DD = pd.read_csv(filename_DD, index_col=None, parse_dates=["datetime_hour"])
df_FB = pd.read_csv(filename_FB, index_col=None, parse_dates=["datetime_hour"])


In [ ]:
df_FB_initial = df_FB.copy()

In [ ]:
df_FB.columns

In [ ]:
df_FB

In [ ]:
# part_1_range_DD = pd.date_range(df_DD.datetime_hour.min(), pd.to_datetime("2024-03-21"), freq="H").tolist()

In [ ]:
train_vali_part_1_range_FB = pd.date_range(df_FB.datetime_hour.min(), pd.to_datetime("2023-07-24"), freq="H")[:-1]

In [ ]:
train_vali_part_1_range_FB

In [ ]:
df_FB.datetime_hour.min()

In [ ]:
df_FB.datetime_hour.max()

In [ ]:
existing_dates_FB = set(df_FB.datetime_hour)

In [ ]:
# existing_dates_DD = set(df_DD.datetime_hour)

In [ ]:
df_FB.datetime_hour.min() in existing_dates_FB

In [ ]:
# for date in part_1_range_DD:
#     if date not in existing_dates_DD:
#         print(f"date {date} missing")

In [ ]:
missing_dates_FB = []
for date in train_vali_part_1_range_FB:
    if date not in existing_dates_FB:
        print(f"date {date} missing")
        missing_dates_FB.append(date)

In [ ]:
missing_dates_FB

In [ ]:
len(missing_dates_FB)

In [ ]:
df_FB_for_interpolation = df_FB[df_FB.datetime_hour.isin(train_vali_part_1_range_FB)].copy()

In [ ]:
df_FB_for_interpolation

In [ ]:
train_vali_part_1_range_FB.max()

In [ ]:
len(df_FB_for_interpolation)

In [ ]:
len(df_FB)

In [ ]:
df_FB.columns

In [ ]:
df_FB.rent_count.mean()

In [ ]:
df_FB.loc[df_FB.datetime_hour.isin(missing_dates_FB), "rent_count"]

In [ ]:
df_FB = df_FB.copy()

In [ ]:
len(df_FB)

In [ ]:
df_FB[['rent_count', "return_count"]].mean()

In [ ]:
df_FB.columns

In [ ]:
for current_cell in df_FB_for_interpolation.hex_id.unique():
    for missing_hour in missing_dates_FB:
        print(f"current_cell {current_cell} missing_hour {missing_hour}")

        hour = missing_hour.hour

        flt_for_interpolation = (df_FB_for_interpolation.hex_id == current_cell) & (df_FB_for_interpolation.datetime_hour.dt.hour == hour)
        values_for_interpolation = df_FB_for_interpolation.loc[flt_for_interpolation, ['rent_count', "return_count"]].mean()
        print(values_for_interpolation)

        flt_to_input = (df_FB.hex_id == current_cell) & (df_FB.datetime_hour == missing_hour)
        # assert such value does not exist
        assert len(df_FB.loc[flt_to_input])==0
        row_to_append = pd.DataFrame({
            "datetime_hour": missing_hour,
            "hex_id": current_cell,
            "rent_count": values_for_interpolation.rent_count,
            "return_count": values_for_interpolation.return_count
        }, index=[0])
        df_FB = pd.concat([df_FB, row_to_append], ignore_index=True)


In [ ]:
len(df_FB)

In [ ]:
assert len(df_FB) - len(df_FB_initial) > 0

In [ ]:
df_FB_part_1 = df_FB.loc[df_FB.datetime_hour < train_vali_part_1_range_FB.max()]

for colname in ['rent_count', "return_count"]:
    for current_cell in df_FB.hex_id.unique():
        plt.figure(figsize=(8, 5))  # 10, 5 was too wide
        flt = (df_FB_part_1.hex_id == current_cell) 
        my_series = df_FB_part_1.loc[flt, [colname, "datetime_hour"]].set_index("datetime_hour")
        sns.lineplot(data=my_series)
        plt.xlabel("Datetime hour")
        plt.ylabel(colname)
        plt.xticks(rotation=90)
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
file_datetime

In [ ]:
time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [ ]:
time

In [ ]:
output_path = f"../data/nextbike/hourly_demand_supply_Freiburg_missing_interpolated_{file_datetime}.csv"
output_path

In [ ]:

df_FB.to_csv(output_path, index=False)